In [24]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [25]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import matplotlib.pyplot as pyplot
import optuna as op
from sklearn.model_selection import ParameterSampler
%matplotlib inline
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Utils.createURM import createURM
from Utils.createICM import createICM

### URM

In [26]:
URM = createURM()
#ICM = createICM()

/Users/matteopancini/PycharmProjects/recsys-challenge-2022-Pancini-Vitali/Utils/createURM.py:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  def createURM():


In [27]:
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.80)

In [28]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 718 ( 1.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 298 ( 0.7%) Users that have less than 1 test interactions


### SLIM

In [29]:
recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train, verbose=False)

def objective(trial):
    alpha = trial.suggest_float("alpha", 0.00001, 0.1)
    l1_ratio = trial.suggest_float("l1_ratio", 0.000001, 0.1)
    topK = trial.suggest_float("topK", 300, 400)
    recommender.fit(alpha=alpha, l1_ratio=l1_ratio, topK=int(topK))

    result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

    return result_dict.loc[10]["MAP"]

In [30]:
study = op.create_study(direction='maximize')
study.optimize(objective, n_trials=3)

[I 2022-11-06 18:50:00,619] A new study created in memory with name: no-name-250fefd3-cd0b-42d6-8989-9f959fa97fe1

100%|█████████▉| 24504/24507 [02:11<00:00, 186.53it/s]


EvaluatorHoldout: Processed 40911 (100.0%) in 18.87 sec. Users per second: 2168


[I 2022-11-06 18:52:30,939] Trial 0 finished with value: 0.02101605575866725 and parameters: {'alpha': 0.04517096810455342, 'l1_ratio': 0.035216709644878094, 'topK': 364.57023977248406}. Best is trial 0 with value: 0.02101605575866725.

100%|█████████▉| 24504/24507 [01:36<00:00, 255.15it/s]


EvaluatorHoldout: Processed 40911 (100.0%) in 16.07 sec. Users per second: 2545


[I 2022-11-06 18:54:23,107] Trial 1 finished with value: 0.020697763204912425 and parameters: {'alpha': 0.035964417388310486, 'l1_ratio': 0.08937450550242909, 'topK': 300.7078115207436}. Best is trial 0 with value: 0.02101605575866725.

100%|█████████▉| 24504/24507 [02:05<00:00, 194.51it/s]


EvaluatorHoldout: Processed 40911 (100.0%) in 18.76 sec. Users per second: 2181


[I 2022-11-06 18:56:47,909] Trial 2 finished with value: 0.020946151789811907 and parameters: {'alpha': 0.015103398776359474, 'l1_ratio': 0.09340180000323595, 'topK': 320.777103313389}. Best is trial 0 with value: 0.02101605575866725.


In [31]:
alpha = study.best_params['alpha']
l1_ratio = study.best_params['l1_ratio']
topK = study.best_params['topK']

In [32]:
alpha

0.04517096810455342

In [33]:
l1_ratio

0.035216709644878094

In [34]:
topK

364.57023977248406

In [35]:
recommender.fit(alpha=alpha, l1_ratio=l1_ratio, topK=int(topK))

result_dict, _ = evaluator_validation.evaluateRecommender(recommender)


100%|█████████▉| 24504/24507 [02:02<00:00, 200.71it/s]


EvaluatorHoldout: Processed 40911 (100.0%) in 19.03 sec. Users per second: 2149


In [48]:
import json

resultParameters = result_dict.to_json(orient="records")
parsed = json.loads(resultParameters)

with open("logs/" + recommender.RECOMMENDER_NAME + "_logs.json", 'w') as json_file:
    json.dump(study.best_params, json_file, indent=4)
    json.dump(parsed, json_file, indent=4)

## Course To Fine

In [ ]:
grid_size = 100
TUNE_ITER = 10
num_epochs = 2
worse_score = 0

init_param_grid = {'l1_ratio': [i for i in range(0, 1)],
                   'topK': [i for i in range(10, 500)]
                  }

new_param_grid = init_param_grid.copy()
best_params_dict = {'score':worse_score,'params':[]}
tried_params_list = []

for epoch in range(num_epochs):

    # List of sampled hyperparameter combinations will be used for random search
    param_list = list(ParameterSampler(new_param_grid, n_iter=TUNE_ITER,random_state=0))

    # Searching the Best Parameters with Random Search
    rs_results_dict = {}
    for tune_iter in range(TUNE_ITER):
        # Get the set of parameter for this iteration
        strategy_params = param_list[tune_iter]

        recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
        recommender.fit(alpha=0.0001, l1_ratio=strategy_params['l1_ratio'], topK=strategy_params['topK'])
        results, _ = evaluator_validation.evaluateRecommender(recommender)
        results = results.loc[10]['MAP']

        rs_results_dict[tuple(strategy_params.values())] = {'score':results}

        if results > best_params_dict['score']:
            best_params_dict['score'] = results
            best_params_dict['params'] = list(strategy_params.values())

    # Save the results in dataframe and sort it based on score
    df_rs_results = pd.DataFrame(rs_results_dict).T.reset_index()
    df_rs_results.columns = list(strategy_params.keys()) + ['score']
    df_rs_results = df_rs_results.sort_values(['score'],ascending=False).head(num_epochs-epoch)

    # If the best score from this epoch is worse than the best score,
    # then append the best hyperaparameters combination to this epoch dataframe
    if df_rs_results['score'].iloc[0] < best_params_dict['score']:
        new_row_dict = {}
        new_row_dict['score'] = best_params_dict['score']
        for idx, key in enumerate(init_param_grid):
            new_row_dict[key] = best_params_dict['params'][idx]

        df_rs_results = df_rs_results.append(new_row_dict,ignore_index=True)
        df_rs_results = df_rs_results.sort_values(['score'],ascending=False).head(num_epochs-epoch)

    display(df_rs_results)
    print(df_rs_results.head(1).T.to_dict())

    # Get the worse and best hyperparameter combinations
    df_rs_results_min = df_rs_results[df_rs_results['score']>worse_score].min(axis=0)
    df_rs_results_max = df_rs_results[df_rs_results['score']>worse_score].max(axis=0)

    # Generate new hyperparameter space based on current worse and best hyperparameter combinations
    for key in init_param_grid:
        if isinstance(init_param_grid[key][0],int):
            new_param_grid[key] = np.unique([i for i in range(int(df_rs_results_min[key]), int(df_rs_results_max[key])+1)])
        elif isinstance(init_param_grid[key][0],float):
            new_param_grid[key] = np.unique(np.linspace(df_rs_results_min[key], df_rs_results_max[key], grid_size))
        else:
            new_param_grid[key] = init_param_grid[key]

    # Decrease the tuning iteration for random search
    TUNE_ITER = int(TUNE_ITER - epoch * TUNE_ITER/num_epochs)

In [ ]:
best_params_dict